In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reasoning Engine in Vertex AI - orchestration with ReAct
@forusone (shins777@gmail.com)

This colab explains how to deal with prompt template for better query and reasoning

References :  
*  https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines
*  https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/develop#prompt-template
*  https://python.langchain.com/v0.2/docs/concepts/#prompt-templates



## Init Vertex AI SDK

In [1]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    "google-cloud-firestore" \
    "langchain-google-firestore" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

In [2]:
import sys
from IPython.display import display, Markdown

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Configuration and initialization

In [3]:
# Static configuration
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-flash"

# Firestore
DATABASE="chat-history"
COLLECTION="history"

### Initialize Vertex AI

In [4]:
import vertexai
from vertexai.preview import reasoning_engines

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Firestore setting for chat history

### Helper method to get session history

In [5]:

# refer to : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/#custom-client

def get_session_history(session_id: str):
    """
    Retrieve chat history data with session id for each conversation from Firestore on GCP
    """

    from langchain_google_firestore import FirestoreChatMessageHistory
    from google.cloud import firestore

    client = firestore.Client(project=PROJECT_ID, database=DATABASE)

    return FirestoreChatMessageHistory(
        client=client,
        session_id=session_id,
        collection=COLLECTION,
        encode_message=False,
    )

def get_converstion(chat_history):
  """
  Get conversation from chat history
  """

  from langchain.schema import HumanMessage, AIMessage
  import langchain_core.messages

  for message in chat_history.messages:
      if type(message) is HumanMessage:
        print(f"Human : {message.content}")
      elif type(message) is AIMessage:
        print(f"AI : {message.content}")

def interactive_chat(agent, query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']


### Unit test of Firestore

In [10]:
# Tool define
from langchain_core.tools import tool

@tool
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
  """
     지정된 날짜의 두 통화 간 환율을 검색합니다.
  """

  import requests

  response = requests.get(
      f"https://api.frankfurter.app/{currency_date}",
      params={"from": currency_from, "to": currency_to},
  )
  return response.json()



In [11]:
print(get_exchange_rate.name)
print(get_exchange_rate.description)
print(get_exchange_rate.args)

get_exchange_rate
지정된 날짜의 두 통화 간 환율을 검색합니다.
{'currency_from': {'default': 'USD', 'title': 'Currency From', 'type': 'string'}, 'currency_to': {'default': 'EUR', 'title': 'Currency To', 'type': 'string'}, 'currency_date': {'default': 'latest', 'title': 'Currency Date', 'type': 'string'}}


In [12]:
from langchain_core.tools import tool

@tool
def get_current_weather(location: str, unit: str = "celsius"):
    """Get the current weather in a given location.
    Args:
        location: The location name.
        unit: The temperature unit.
    Returns:
        The current weather.
    """
    return "26c"

In [13]:
# Let's inspect some of the attributes associated with the tool.
print(get_current_weather.name)
print(get_current_weather.description)
print(get_current_weather.args)

get_current_weather
Get the current weather in a given location.
    Args:
        location: The location name.
        unit: The temperature unit.
    Returns:
        The current weather.
{'location': {'title': 'Location', 'type': 'string'}, 'unit': {'default': 'celsius', 'title': 'Unit', 'type': 'string'}}


In [14]:
from typing import Sequence
from langchain_core.language_models import BaseLanguageModel
from langchain_core.prompts import BasePromptTemplate
from langchain_core.tools import BaseTool
from langchain import hub
from langchain_core.prompts import PromptTemplate

# from vertexai.generative_models import FunctionDeclaration
# from vertexai.generative_models import Tool

def react_builder(
    model: BaseLanguageModel,
    *,
    tools: Sequence[BaseTool],
    prompt: BasePromptTemplate,
    agent_executor_kwargs = None,
    **kwargs,
):
    from langchain.agents.react.agent import create_react_agent
    from langchain.agents import AgentExecutor

    agent = create_react_agent(model, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, **agent_executor_kwargs)

template = """
  Answer the following questions as best you can. You have access to the following tools:

  {tools}

  Use the following format:

  Question: the input question you must answer
  Thought: you should always think about what to do
  Action: the action to take, should be one of [{tool_names}]
  Action Input: the input to the action
  Observation: the result of the action
  ... (this Thought/Action/Action Input/Observation can repeat N times)
  Thought: I now know the final answer
  Final Answer: the final answer to the original input question

  Begin!

  Question: {input}
  Thought: {agent_scratchpad}
"""

prompt = PromptTemplate.from_template(template)

agent = reasoning_engines.LangchainAgent(
    model=MODEL,
    tools=[get_current_weather, get_exchange_rate],
    prompt = prompt,
    agent_executor_kwargs={"verbose": True}, # Optional. For illustration.
    runnable_builder=react_builder,
)

In [ ]:
# response = agent.query(input={"role": "user", "content": "원달러 환율에 대해서 알려주세요."})
response = agent.query(input={"input": "2024-11-13 날짜의  달러 환율에 대해서 알려주세요."})
# response = agent.query(input={"input": "서울 날씨는 섭씨 몇도인가요 ?",
#                               "tool_names":"get_current_weather, get_exchange_rate"
#                               })
